In [ ]:
import torch

def cartesian_to_polar_torch(matrix):
    """
    Converts each row of a Torch tensor from Cartesian coordinates to polar coordinates.

    Args:
        matrix (torch.Tensor): An m x n tensor where each row is an n-dimensional Cartesian vector.

    Returns:
        torch.Tensor: An m x n tensor where each row is the polar coordinates of the corresponding Cartesian vector.
    """
    polar_coordinates = []

    for row in matrix:
        # Compute the radial distance r
        r = torch.norm(row)

        # Compute the angles
        angles = []
        for i in range(len(row) - 1):
            denominator = torch.sqrt(torch.sum(row[i:] ** 2))
            if denominator == 0:
                angle = 0
            else:
                angle = torch.acos(row[i] / denominator)
            angles.append(angle)

        # Add the azimuthal angle for the last component
        if len(row) > 1:
            azimuth = torch.atan2(row[-1], row[-2])
            angles[-1] = azimuth  # Replace the last angle with azimuth

        polar_coordinates.append(torch.tensor([r] + angles))

    return torch.stack(polar_coordinates)

def polar_to_cartesian_torch(polar_matrix):
    """4
    Converts each row of a Torch tensor from polar coordinates to Cartesian coordinates.

    Args:
        polar_matrix (torch.Tensor): An m x n tensor where each row is the polar coordinates.

    Returns:
        torch.Tensor: An m x n tensor where each row is the Cartesian coordinates of the corresponding polar vector.
    """
    cartesian_coordinates = []

    for row in polar_matrix:
        r = row[0]
        angles = row[1:]

        # Initialize Cartesian coordinates
        cartesian = torch.zeros_like(angles)
        cartesian = torch.cat([torch.tensor([r]), cartesian])

        # Compute Cartesian coordinates
        for i in range(len(angles)):
            cartesian[i] = r * torch.prod(torch.sin(angles[:i])) * torch.cos(angles[i])

        cartesian[-1] = r * torch.prod(torch.sin(angles))  # Last component
        cartesian_coordinates.append(cartesian)

    return torch.stack(cartesian_coordinates)


def pseudo_quantize_tensor(
    w, n_bit=8, zero_point=True, q_group_size=-1, inplace=False, get_scale_zp=False
):
    org_w_shape = w.shape
    if q_group_size > 0:
        assert org_w_shape[-1] % q_group_size == 0
        w = w.reshape(-1, q_group_size)
    assert w.dim() == 2
    if zero_point:
        max_val = w.amax(dim=1, keepdim=True)
        min_val = w.amin(dim=1, keepdim=True)
        max_int = 2**n_bit - 1
        min_int = 0
        scales = (max_val - min_val).clamp(min=1e-5) / max_int
        zeros = (-torch.round(min_val / scales)).clamp_(min_int, max_int)
    else:  # we actually never used this
        assert min_val is None
        max_val = w.abs().amax(dim=1, keepdim=True)
        max_val = max_val.clamp(min=1e-5)
        max_int = 2 ** (n_bit - 1) - 1
        min_int = -(2 ** (n_bit - 1))
        scales = max_val / max_int
        zeros = 0

    assert torch.isnan(scales).sum() == 0
    assert torch.isnan(w).sum() == 0

    if inplace:
        (
            (w.div_(scales).round_().add_(zeros)).clamp_(min_int, max_int).sub_(zeros)
        ).mul_(scales)
    else:
        w = (
            torch.clamp(torch.round(w / scales) + zeros, min_int, max_int) - zeros
        ) * scales
    assert torch.isnan(w).sum() == 0

    w = w.reshape(org_w_shape)

    if get_scale_zp:
        return w, scales.view(w.shape[0], -1), zeros.view(w.shape[0], -1)
    else:
        return w

In [26]:
mse_fn = torch.nn.MSELoss()
m = torch.tensor([
    [100, -6.0, 2.0],
    [3.0, -3.0, 100.0],
    [0.0, -300.0, 1.0]
])

print(m)

m_q = pseudo_quantize_tensor(m)
print(m_q)
print(mse_fn(m, m_q))

m_p = cartesian_to_polar_torch(m)
print(m_p)
m_p = pseudo_quantize_tensor(m_p.T)
m_p = polar_to_cartesian_torch(m_p.T)

print(m_p)
print(mse_fn(m, m_p))

tensor([[ 100.,   -6.,    2.],
        [   3.,   -3.,  100.],
        [   0., -300.,    1.]])
tensor([[ 100.1804,   -5.8196,    2.0784],
        [   2.8275,   -2.8275,  100.1726],
        [   0.0000, -299.8196,    1.1804]])
tensor(0.0251)
tensor([[ 1.0020e+02,  9.9801e-01, -9.4868e-01,  1.0000e+00],
        [ 1.0009e+02,  2.9973e-02, -2.9987e-02,  1.0000e+00],
        [ 3.0000e+02,  0.0000e+00, -9.9999e-01,  1.0000e+00]])
tensor([[ 100.1477,   -5.9993,    2.0312],
        [   3.1419,   -3.0523,  100.2522],
        [   0.0000, -193.9159,   48.5932]])
tensor(1502.1167)


In [ ]:
# Example usage
matrix = torch.tensor([
    [100, -2.0, 2.0],
    [3.0, -3.0, 3.0],
    [0.0, -1.0, 0.0]
])

# Convert to polar coordinates
polar_coords = cartesian_to_polar_torch(matrix)
print("Polar Coordinates:")
print(polar_coords)

# Convert back to Cartesian coordinates
restored_matrix = polar_to_cartesian_torch(polar_coords)
print("Restored Cartesian Coordinates:")
print(restored_matrix)

# Verify reconstruction
print("Reconstruction successful:", torch.allclose(matrix, restored_matrix, atol=1e-6))

In [23]:
import torch

def cartesian_to_polar_torch(matrix):
    """
    Converts each row of a Torch tensor from Cartesian coordinates to polar coordinates.

    Args:
        matrix (torch.Tensor): An m x n tensor where each row is an n-dimensional Cartesian vector.

    Returns:
        torch.Tensor: An m x (n+1) tensor where each row contains the radial distance (r) and cosines of the angles.
    """
    polar_coordinates = []

    for row in matrix:
        # Compute the radial distance r
        r = torch.norm(row)

        # Compute cosines of the angles
        cos_values = []
        for i in range(len(row)):
            denominator = torch.sqrt(torch.sum(row[i:] ** 2)) + 1e-20  # Avoid division by zero
            cos_values.append(row[i] / denominator)

        polar_coordinates.append(torch.cat([torch.tensor([r]), torch.tensor(cos_values)]))

    return torch.stack(polar_coordinates)

def polar_to_cartesian_torch(polar_matrix):
    """
    Converts each row of a Torch tensor from stored polar representation back to Cartesian coordinates.

    Args:
        polar_matrix (torch.Tensor): An m x (n+1) tensor where each row contains the radial distance (r) and cosines of the angles.

    Returns:
        torch.Tensor: An m x n tensor where each row is the restored Cartesian vector.
    """
    cartesian_coordinates = []

    for row in polar_matrix:
        # Extract radial distance and cosine values
        r = row[0]
        cos_values = row[1:]
        
        # Initialize an empty Cartesian vector
        cartesian = torch.zeros_like(cos_values)
        
        # Restore Cartesian coordinates iteratively
        remaining_r = r
        for i in range(len(cos_values) - 1):
            cartesian[i] = remaining_r * cos_values[i]
            remaining_r = remaining_r * torch.sqrt(1 - cos_values[i]**2)
        cartesian[-1] = remaining_r  # Last component
        
        cartesian_coordinates.append(cartesian)

    return torch.stack(cartesian_coordinates)

# Example usage
matrix = torch.tensor([
    [1.0, 2.0, 2.0],
    [3.0, 3.0, 3.0],
    [0.0, 1.0, 0.0]
])

# Convert to polar-like coordinates (storing r and cosines of angles)
polar_coords = cartesian_to_polar_torch(matrix)
print("Polar-Like Coordinates (r and cosines):")
print(polar_coords)

# Convert back to Cartesian coordinates
restored_matrix = polar_to_cartesian_torch(polar_coords)
print("Restored Cartesian Coordinates:")
print(restored_matrix)

# Verify reconstruction
print("Reconstruction successful:", torch.allclose(matrix, restored_matrix, atol=1e-6))


Polar-Like Coordinates (r and cosines):
tensor([[3.0000, 0.3333, 0.7071, 1.0000],
        [5.1962, 0.5774, 0.7071, 1.0000],
        [1.0000, 0.0000, 1.0000, 0.0000]])
Restored Cartesian Coordinates:
tensor([[1.0000, 2.0000, 2.0000],
        [3.0000, 3.0000, 3.0000],
        [0.0000, 1.0000, 0.0000]])
Reconstruction successful: True


In [28]:
import torch

def cartesian_to_polar_torch(matrix):
    """
    Converts each row of a Torch tensor from Cartesian coordinates to polar coordinates.

    Args:
        matrix (torch.Tensor): An m x n tensor where each row is an n-dimensional Cartesian vector.

    Returns:
        torch.Tensor: An m x (n+1) tensor where each row contains the radial distance (r) and cosines of the angles.
    """
    polar_coordinates = []

    for row in matrix:
        # Compute the radial distance r
        r = torch.norm(row)

        # Compute cosines of the angles
        cos_values = []
        for i in range(len(row)):
            denominator = torch.sqrt(torch.sum(row[i:] ** 2)) + 1e-8  # Avoid division by zero
            cos_values.append(row[i] / denominator)

        polar_coordinates.append(torch.cat([torch.tensor([r]), torch.tensor(cos_values)]))

    return torch.stack(polar_coordinates)

def polar_to_cartesian_torch(polar_matrix):
    """
    Converts each row of a Torch tensor from stored polar representation back to Cartesian coordinates.

    Args:
        polar_matrix (torch.Tensor): An m x (n+1) tensor where each row contains the radial distance (r) and cosines of the angles.

    Returns:
        torch.Tensor: An m x n tensor where each row is the restored Cartesian vector.
    """
    cartesian_coordinates = []

    for row in polar_matrix:
        # Extract radial distance and cosine values
        r = row[0]
        cos_values = row[1:]

        # Restore Cartesian components iteratively
        cartesian = torch.zeros_like(cos_values)
        remaining_r = r
        for i in range(len(cos_values) - 1):
            cartesian[i] = remaining_r * cos_values[i]
            remaining_r = remaining_r * torch.sqrt(1 - cos_values[i]**2)
        cartesian[-1] = remaining_r  # Last component

        cartesian_coordinates.append(cartesian)

    return torch.stack(cartesian_coordinates)

def compare_dot_product(matrix_a, matrix_b):
    """
    Compare dot products in Cartesian and Polar coordinate systems.

    Args:
        matrix_a (torch.Tensor): Tensor of shape (m, n) representing vectors in Cartesian coordinates.
        matrix_b (torch.Tensor): Tensor of shape (m, n) representing vectors in Cartesian coordinates.

    Returns:
        None
    """
    # Cartesian dot product
    cartesian_dot_products = torch.sum(matrix_a * matrix_b, dim=1)

    # Convert to polar coordinates
    polar_a = cartesian_to_polar_torch(matrix_a)
    polar_b = cartesian_to_polar_torch(matrix_b)

    # Polar dot product: r1 * r2 * cos(theta_diff)
    # Polar dot product: r1 * r2 * cos(theta_diff)
    polar_dot_products = []
    for a, b in zip(polar_a, polar_b):
        r1, cos_values_a = a[0], a[1:]
        r2, cos_values_b = b[0], b[1:]
        
        # Initialize cosine of the angle difference
        cos_theta_diff = 1.0
        remaining_r1, remaining_r2 = r1, r2

        for i in range(len(cos_values_a) - 1):
            # Update the cosine difference for each dimension
            cos_theta_diff *= cos_values_a[i] * cos_values_b[i]
            remaining_r1 *= torch.sqrt(1 - cos_values_a[i]**2)
            remaining_r2 *= torch.sqrt(1 - cos_values_b[i]**2)
        
        # Include the last dimension
        cos_theta_diff *= (remaining_r1 * remaining_r2) / (r1 * r2)
        polar_dot_products.append(r1 * r2 * cos_theta_diff)

    polar_dot_products = torch.tensor(polar_dot_products)


    polar_dot_products = torch.tensor(polar_dot_products)

    # Print results
    print("Cartesian Dot Products:")
    print(cartesian_dot_products)
    print("Polar Dot Products:")
    print(polar_dot_products)

    # Verify closeness
    print("Results are close:", torch.allclose(cartesian_dot_products, polar_dot_products, atol=1e-6))

# Example usage
matrix_a = torch.tensor([
    [1.0, 2.0, 2.0],
    [3.0, 3.0, 3.0],
    [0.0, 1.0, 0.0]
])

matrix_b = torch.tensor([
    [2.0, 1.0, 0.0],
    [1.0, 2.0, 1.0],
    [1.0, 0.0, 1.0]
])

compare_dot_product(matrix_a, matrix_b)


Cartesian Dot Products:
tensor([ 4., 12.,  0.])
Polar Dot Products:
tensor([   nan, 0.4472,    nan])
Results are close: False


/tmp/ipykernel_3244955/3031421043.py:100: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  polar_dot_products = torch.tensor(polar_dot_products)
